# SQL Database Agent

In [ ]:
import os
import openai
from IPython.display import display, HTML, Markdown
from pprint import pprint

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
from langchain.callbacks import OpenAICallbackHandler

totals_cb = OpenAICallbackHandler()

print(totals_cb)

### Create an example SQL Database

This notebook uses an example SQL Database implemented with SQLite3.

Database schema and data are created using scripts from [The Sakila example database repo](https://github.com/jOOQ/sakila)

[Sakila docs at dev.mysql.com](https://dev.mysql.com/doc/sakila/en/)

In [ ]:
!rm -rf sakila
!git clone https://github.com/jOOQ/sakila

In [ ]:
%%bash
rm -f sqlite-sakila.db
cat << EOF | sqlite3
.open sqlite-sakila.db
.read sakila/sqlite-sakila-db/sqlite-sakila-schema.sql
.read sakila/sqlite-sakila-db/sqlite-sakila-insert-data.sql
EOF
ls -l sqlite-sakila.db

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

db = SQLDatabase.from_uri("sqlite:///sqlite-sakila.db")
toolkit = SQLDatabaseToolkit(db=db, llm=ChatOpenAI(model="gpt-4", temperature=0))

In [ ]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import MessagesPlaceholder

memory = ConversationBufferWindowMemory(k=3, memory_key="chat_history", return_messages=True)

agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="chat_history")],
}

`create_sql_agent` [API reference](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.sql.base.create_sql_agent.html)

In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_types import AgentType

agent_executor_kwargs = {
    "handle_parsing_errors": True
}

agent = create_sql_agent(
    llm=ChatOpenAI(model="gpt-4", temperature=0.0),
    toolkit=toolkit,
    memory=memory,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    agent_executor_kwargs=agent_executor_kwargs,
)

In [ ]:
response = agent.run("Describe tables", callbacks=[totals_cb])

display(Markdown(response))

In [ ]:
response = agent.run("Show film categories", callbacks=[totals_cb])

display(Markdown(response))

In [ ]:
response = agent.run("Which films are currently rented?", callbacks=[totals_cb])

display(Markdown(response))

In [ ]:
response = agent.run("Who has rented film HYDE DOCTOR most recently?", callbacks=[totals_cb])

display(Markdown(response))

In [ ]:
print(totals_cb)